In [100]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [101]:
###data 읽기
young = pd.read_csv('./youngpeoplesurvey/responses.csv')
young.head(5);

In [102]:
###null값이 포함된 데이터
youngwhnull = young[young[['Weight','Height']].isnull().any(axis=1)]
len(youngwhnull.index);
youngwhnull.head();
###weight와 height의 null값을 제외한 데이터
youngwh = young[young[['Weight','Height']].notnull().all(axis=1)]
youngwh.head();
len(youngwh.index);
###총데이터 : 1010 확인
print(len(young.index) == len(youngwh) + len(youngwhnull))
print(len(young))

True
1010


In [103]:
###bmi 항목 추가
young = youngwh
young['bmi'] = young['Weight']/((young['Height']/100.0)**2)
young.head();

In [104]:
###outlier 제거(height, weight thresholding)
young = young.loc[(young['Height']>142.42) & (young['Height']<195.58) & (young['Weight']>44.45) & (young['Weight']<145)]
print(len(young.index));

965


In [105]:
###outlier 제거(number of siblings & only child 모순 1)
mask = (young['Number of siblings'] == 0) & (young['Only child'] == 'no')
col1 = 'Number of siblings'
col2 = 'Only child'
young.loc[[298, 320, 626, 925, 954]];
young.loc[mask.index[mask == True], col1] = np.nan
young.loc[mask.index[mask == True], col2] = np.nan
print(len(young.loc[(young['Number of siblings'] == 0) & (young['Only child'] == 'no')]))
young.loc[[298, 320, 626, 925, 954]];

0


C:\Users\DGIST-VL\Anaconda2\envs\py3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [106]:
###outlier 제거(number of siblings & only child 모순 2)
mask2 = (young['Number of siblings'] != 0) & (young['Only child'] == 'yes')
young.loc[mask2.index[mask2 == True], 'Number of siblings'] = np.nan
young.loc[mask2.index[mask2 == True], 'Only child'] = np.nan
print(len(young.loc[(young['Number of siblings'] != 0) & (young['Only child'] == 'yes')]))

0


C:\Users\DGIST-VL\Anaconda2\envs\py3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [107]:
###categoical data를 1,2,3,4,5로 변경
####타입 변경
cat_columns = young.select_dtypes(['object']).columns
print(cat_columns);
for col in cat_columns:
    young[col] = young[col].astype(dtype='category')
####category order 변수 생성
sorter_dic = {"Smoking" : "Never smoked - Tried smoking - Former smoker - Current smoker",
             "Alcohol" : "Never - Social drinker - Drink a lot",
             "Punctuality" : "I am often early - I am always on time - I am often running late",
             "Lying" : "Never - Only to avoid hurting someone - Sometimes - Everytime it suits me",
             "Internet usage" : "No time at all - Less than an hour a day - Few hours a day - Most of the day",
             "Gender" : "Female - Male",
             "Left - right handed" : "Left handed - Right handed",
             "Education" : "Currently a Primary school pupil - Primary school - Secondary school - College/Bachelor degree - Masters degree - doctorate degree",
             "Only child" : "No - Yes",
             "Village - town" : "City - village",
             "House - block of flats" : "house/bungalow - block of flats"     
             }
for key, value in sorter_dic.items():
    sorter_dic[key] = value.lower().split(' - ')
print(sorter_dic)

Index(['Smoking', 'Alcohol', 'Punctuality', 'Lying', 'Internet usage',
       'Gender', 'Left - right handed', 'Education', 'Only child',
       'Village - town', 'House - block of flats'],
      dtype='object')


C:\Users\DGIST-VL\Anaconda2\envs\py3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


{'Smoking': ['never smoked', 'tried smoking', 'former smoker', 'current smoker'], 'Alcohol': ['never', 'social drinker', 'drink a lot'], 'Punctuality': ['i am often early', 'i am always on time', 'i am often running late'], 'Lying': ['never', 'only to avoid hurting someone', 'sometimes', 'everytime it suits me'], 'Internet usage': ['no time at all', 'less than an hour a day', 'few hours a day', 'most of the day'], 'Gender': ['female', 'male'], 'Left - right handed': ['left handed', 'right handed'], 'Education': ['currently a primary school pupil', 'primary school', 'secondary school', 'college/bachelor degree', 'masters degree', 'doctorate degree'], 'Only child': ['no', 'yes'], 'Village - town': ['city', 'village'], 'House - block of flats': ['house/bungalow', 'block of flats']}


In [108]:
####Category order에 맞게 new value 부여
for col in cat_columns:
    young[col] = young[col].cat.reorder_categories(sorter_dic[col])
for col in cat_columns:
    young[col] = young[col].cat.codes
####1부터 시작
young[cat_columns] = young[cat_columns] + 1

C:\Users\DGIST-VL\Anaconda2\envs\py3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\DGIST-VL\Anaconda2\envs\py3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\DGIST-VL\Anaconda2\envs\py3\lib\site-packages\pandas\core\frame.py:2450: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [109]:
###Bmi_class 추가
young["bmi_class"]=1
young.loc[((young["bmi"]>18.5) & (young["bmi"]<25)),"bmi_class"]=2
young.loc[((young["bmi"]>25) & (young["bmi"]<30)),"bmi_class"]=3
young.loc[(young["bmi"]>30),"bmi_class"]=4
### 갯수 확인
print(len(young.loc[young["bmi_class"]==1]))
print(len(young.loc[young["bmi_class"]==2]))
print(len(young.loc[young["bmi_class"]==3]))
print(len(young.loc[young["bmi_class"]==4]))
print(len(young))

108
723
122
12
965


C:\Users\DGIST-VL\Anaconda2\envs\py3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\DGIST-VL\Anaconda2\envs\py3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [110]:
# ### 하나라도 null이 있는 column없애기
young_cp = young.copy()
for feature in young_cp.columns:
    young_cp = young_cp[~young_cp[feature].isnull()]
### 갯수 확인
print(len(young_cp.loc[young_cp["bmi_class"]==1]))
print(len(young_cp.loc[young_cp["bmi_class"]==2]))
print(len(young_cp.loc[young_cp["bmi_class"]==3]))
print(len(young_cp.loc[young_cp["bmi_class"]==4]))
print(len(young_cp))

74
452
77
10
613


In [111]:
###Test_set 뽑기 
young_train = young.copy()
test_class_1 = young_cp.loc[young_cp["bmi_class"]==1].sample(n=3); young_train = young_train.drop(test_class_1.index)
print(len(young_train));
test_class_2 = young_cp.loc[young_cp["bmi_class"]==2].sample(n=3); young_train = young_train.drop(test_class_2.index)
print(len(young_train));
test_class_3 = young_cp.loc[young_cp["bmi_class"]==3].sample(n=3);  young_train = young_train.drop(test_class_3.index)
print(len(young_train));
test_class_4 = young_cp.loc[young_cp["bmi_class"]==4].sample(n=3);  young_train = young_train.drop(test_class_4.index)
print(len(young_train));
####test_set 쌓기
test_list = [test_class_1, test_class_2, test_class_3, test_class_4]
test = pd.concat(test_list)
### test 가 제대로 생성되었는 지 확인
print(test.isnull().sum().sum())
print(young_train.isnull().sum().sum())
print(young.isnull().sum().sum())

962
959
956
953
0
593
593


In [112]:
###bmi_class binary version
young_binary = young.copy()
young_binary.loc[(young_binary["bmi_class"]==2),"bmi_class"]=1
young_binary.loc[(young_binary["bmi_class"]==3),"bmi_class"]=1
young_binary.loc[(young_binary["bmi_class"]==4),"bmi_class"]=2
print(len(young_binary.loc[young_binary['bmi_class']==2]))
# for feature in ['Gender', 'Age', 'Shopping', 'PC', 'Life struggles', 'Romantic']:
#     youngbmit = youngbmit[~youngbmit[feature].isnull()]
young_binary.to_csv("young_bianry.csv")

12


In [89]:
###Train data 생성
train = young_train.copy()
print(len(young_train))
print(len(young))
# train = train[['Gender', 'Age', 'Shopping', 'PC', 'Life struggles', 'Romantic', 'bmi_class']]
# test = test[['Gender', 'Age', 'Shopping', 'PC', 'Life struggles', 'Romantic', 'bmi_class']]
train.to_csv("young_train_25_binary_pure.csv")
test.to_csv("young_test_balanced_25_binary.csv")